# I. Data Loading and Preprocessing

Data source from `Dreaddit: A Reddit Dataset for Stress Analysis in Social Media`.

Following tasks are undertaken:
* Columns Selection
* Feature Transformation
* Handling Missing Values
* Column Encoding

## Categorical Data Encoding, Feature Selection and Training...

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

%matplotlib inline
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")


In [2]:
_pipeline_name = 'redmodel'

_mh_root = os.path.join(os.environ['HOME'], 'redmodel')
_data_root = os.path.join(_mh_root, 'data', 'simple')
# Python module file to inject customized logic into the TFX components. The
# Transform and Trainer both require user-defined functions to run successfully.
_module_file = os.path.join(_mh_root, 'taxi_utils.py')
# Path which can be listened to by the model server.  Pusher will output the
# trained model here.
_serving_model_dir = os.path.join(_mh_root, 'serving_model', _pipeline_name)

# Directory and data locations.  This example assumes all of the chicago taxi
# example code and metadata library is relative to $HOME, but you can store
# these files anywhere on your local filesystem.
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_root = os.path.join(_mh_root, 'pipelines', _pipeline_name)
# Sqlite ML-metadata db path.
_metadata_path = os.path.join(_mh_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [4]:
# Data import
dreaddit_train_df = pd.read_csv(_data_root+"/dreaddit-train.csv")
dreaddit_test_df = pd.read_csv(_data_root+"/dreaddit-test.csv")
tweet_unsupervised_df = pd.read_csv(_data_root+"/tweet-unsupervised.csv")

In [5]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_" + metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_" + metric])

In [6]:
dreaddit_train_df.head()

,subreddit,post_id,sentence_range,text,id,label,confidence,social_timestamp,social_karma,syntax_ari,...,lex_dal_min_pleasantness,lex_dal_min_activation,lex_dal_min_imagery,lex_dal_avg_activation,lex_dal_avg_imagery,lex_dal_avg_pleasantness,social_upvote_ratio,social_num_comments,syntax_fk_grade,sentiment
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",33181,1,0.8,1521614353,5,1.806818,...,1.000,1.1250,1.0,1.77000,1.52211,1.89556,0.86,1,3.253573,-0.002742
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",2606,0,1.0,1527009817,4,9.429737,...,1.125,1.0000,1.0,1.69586,1.62045,1.88919,0.65,2,8.828316,0.292857
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,38816,1,0.8,1535935605,2,7.769821,...,1.000,1.1429,1.0,1.83088,1.58108,1.85828,0.67,0,7.841667,0.011894
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",239,1,0.6,1516429555,0,2.667798,...,1.000,1.1250,1.0,1.75356,1.52114,1.98848,0.50,5,4.104027,0.141671
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1421,1,0.8,1539809005,24,7.554238,...,1.000,1.1250,1.0,1.77644,1.64872,1.81456,1.00,1,7.910952,-0.204167


In [8]:
tweet_unsupervised_df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,���It felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [9]:
print(dreaddit_test_df.text[5])

Thanks. Edit 1 - Fuel Receipt As Requested. <url> Sorry for the long responses, I went to spend the night at a friends because it got really cold here! The Police said they don't give out a copy of the report but they gave me an incident number that can be used to verify the report was filed.


In [10]:
cols = dreaddit_train_df.columns

In [11]:
new_cols = [
    "text",
    "label",
]

In [12]:
train_df = dreaddit_train_df[new_cols]
train_df.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


In [13]:
test_cols = dreaddit_test_df.columns
dreaddit_test_df.info()
test_df = dreaddit_test_df[new_cols]
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Columns: 116 entries, id to sentiment
dtypes: float64(107), int64(5), object(4)
memory usage: 648.1+ KB


,text,label
0,"Its like that, if you want or not.“ ME: I have...",0
1,I man the front desk and my title is HR Custom...,0
2,We'd be saving so much money with this new hou...,1
3,"My ex used to shoot back with ""Do you want me ...",1
4,I haven’t said anything to him yet because I’m...,0


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2838 non-null   object
 1   label   2838 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 44.5+ KB


In [17]:
text_col = [
    "text"
]
unsup_df = tweet_unsupervised_df[text_col]

In [18]:
unsup_df.head()

,text
0,Robbie E Responds To Critics After Win Against...
1,���It felt like they were my friends and I was...
2,i absolutely adore when louis starts the songs...
3,Hi @JordanSpieth - Looking at the url - do you...
4,Watching Neighbours on Sky+ catching up with t...


In [19]:
unsup_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20050 non-null  object
dtypes: object(1)
memory usage: 156.8+ KB


In [16]:
train_df.label.value_counts()

1    1488
0    1350
Name: label, dtype: int64

In [28]:
train_df.to_csv(r"../data/final/train.csv")
test_df.to_csv(r"../data/final/eval/test.csv")
unsup_df.to_csv(r"../data/final/unsup.csv")

# III. TF Feature Columns to Classify Structure Data

In [1]:
import os
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow import keras

import numpy as np

## Create an input pipeline using tf.data

In [15]:
# Convert to tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("label")
    ds = tf.data.Dataset.from_tensor_slices((dataframe, labels))
    return ds

In [16]:
train_ds = df_to_dataset(train_df)
test_ds = df_to_dataset(test_df)

In [17]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [18]:
train_dataset = (
    train_ds.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
test_dataset = test_ds.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [19]:
train_dataset.element_spec

(TensorSpec(shape=(None, 1), dtype=tf.string, name=None),
 TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE
)
encoder.adapt(train_ds.map(lambda text, label: text))

In [22]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'i', 'to', 'and', 'the', 'a', 'my', 'of', 'me',
       'that', 'in', 'it', 'for', 'was', 'is', 'but', 'have', 'with',
       'this'], dtype='<U13')

In [23]:
for example, label in train_dataset.take(1):
    print("texts: ", example.numpy()[:1])
    print()
    print("labels: ", label.numpy()[:1])

texts:  [[b"They were too caught up in their anger. My grandmother would tell me stories about having to be the one that cleaned the blood from the walls. She thought I might be mentally handicapped because I wouldn't make a sound when all of this was going on. My brother was three years older than I was. My mother left my father when I was two."]]

labels:  [0]


In [24]:
encoded_example = encoder(example)[:1].numpy()
encoded_example

array([[ 58,  92, 138,   1,  39,  11, 141, 767,   7,   1,  54, 175,   9,
          1,  30, 142,   3,  27,   5,  67,  10,   1,   5, 755,  50,   5,
          1,  23, 167,   2, 274,  27, 723,   1,  43,   2, 534, 116,   6,
        817,  44,  48,   8,  19,  14,  81,  21,   7, 382,  14, 434,  83,
        549, 132,   2,  14,   7, 302, 284,   7, 348,  44,   2,  14, 214,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

In [25]:
model = tf.keras.Sequential(
    [
        encoder,
        tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1),
    ]
)

In [29]:
sample_text = "I have been experiencing a depressing mood and my stress levels are rising tremendously. This should indicate something"
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00219835]


In [30]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1
)
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


NameError: name 'datetime' is not defined

In [31]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"],
)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          64000     
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [27]:
history = model.fit(
    train_dataset,
    epochs=10,
    callbacks=[cp_callback, tensorboard_callback],
    validation_data=test_dataset,
    validation_steps=30,
)

Epoch 1/10
45/45 [==============================] - ETA: 0s - loss: 0.6924 - accuracy: 0.4757WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 30 batches). You may need to use the repeat() function when building your dataset.

Epoch 00001: saving model to training_1/cp.ckpt
45/45 [==============================] - 50s 1s/step - loss: 0.6924 - accuracy: 0.4757 - val_loss: 0.6920 - val_accuracy: 0.4839
Epoch 2/10
45/45 [==============================] - ETA: 0s - loss: 0.6899 - accuracy: 0.4757
Epoch 00002: saving model to training_1/cp.ckpt
45/45 [==============================] - 36s 798ms/step - loss: 0.6899 - accuracy: 0.4757
Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 0.6787 - accuracy: 0.4838
Epoch 00003: saving model to training_1/cp.ckpt
45/45 [==============================] - 36s 796ms/step - loss: 0.6787 - accuracy: 0.4838

In [28]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save("saved_model/red_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/red_model/assets


In [29]:
test_loss, test_acc = model.evaluate(test_dataset)

print("Test Loss: {}".format(test_loss))
print("Test Accuracy: {}".format(test_acc))

12/12 [==============================] - 3s 259ms/step - loss: 0.5625 - accuracy: 0.7175
Test Loss: 0.5625007152557373
Test Accuracy: 0.7174825072288513


In [30]:
# Predict on a sample to test accuract and padding.

sample_text = "I have been experiencing a depressing mood and my stress levels are rising tremendously. This should indicate something."
predictions = model.predict(np.array([sample_text]))
print(predictions)

[[-0.84668046]]


In [31]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

%tensorboard --logdir logs/scalars

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 4709.